In [ ]:
#use python 3.9.13, as that is geared exclusively for this package now and doesn't have numpy

In [ ]:
%gui qt

import sys
import struct
from PySide6.QtWidgets import QApplication
from SquidstatPyLibrary import AisDeviceTracker
from SquidstatPyLibrary import AisCompRange
from SquidstatPyLibrary import AisDCData
from SquidstatPyLibrary import AisACData
from SquidstatPyLibrary import AisExperimentNode
from SquidstatPyLibrary import AisErrorCode
from SquidstatPyLibrary import AisExperiment
import numpy as np


from SquidstatPyLibrary import AisConstantPotElement
from SquidstatPyLibrary import AisEISPotentiostaticElement
from SquidstatPyLibrary import AisConstantCurrentElement
from SquidstatPyLibrary import (
    AisDeviceTracker, AisCompRange, AisDCData, AisACData, AisExperimentNode,
    AisErrorCode, AisExperiment, AisInstrumentHandler,
    AisConstantPotElement, AisEISPotentiostaticElement, AisConstantCurrentElement, AisCyclicVoltammetryElement
)

tracker = AisDeviceTracker.Instance()
tracker.connectToDeviceOnComPort("COM7")

In [9]:
def record_cv(deviceName, low, high, end, step = 0.01, sampling_interval = 0.01, start = 0, repeat = 3):
    print("Device is Connected: %s" % deviceName)
    
    handler = tracker.getInstrumentHandler("Prime2645")

    # Connect signals; assigning the connection to "_" prevents printing the connection object.
    handler.activeDCDataReady.connect(lambda channel, data: print(f"timestamp:", "{:.3f},".format(data.timestamp),
                                                                  'voltage:', '{:.9f},'.format(data.counterElectrodeVoltage),
                                                                  "working Electrode Voltage:","{:.9f},".format(data.workingElectrodeVoltage),
                                                                  'counter electrode:', '{:.9f},'.format(data.counterElectrodeVoltage),
                                                                  'current:', "{:.9f}".format(data.current)))
    handler.experimentNewElementStarting.connect(lambda channel, data: print(f"New Node beginning:", data.stepName, "step number: ", data.stepNumber, " step sub : ", data.substepNumber))
    handler.experimentStopped.connect(lambda channel : print(f"Experiment Completed: %d" % channel))

    # Set up experiment elements
    experiment = AisExperiment()
    
    # define cyclic voltammetry between starting at 0V, scanning between -0.25 and 0.25 V, ending at 0.25 V with a step size of 0.1 V/sec and sampling every 0.1 s
    cyclic = AisCyclicVoltammetryElement(start, low, high, end, step, sampling_interval)

    success = False
    subExperiment = AisExperiment()
    success &= subExperiment.appendElement(cyclic)
    success &= experiment.appendSubExperiment(subExperiment, repeat)

    error = handler.uploadExperimentToChannel(0, experiment)
    if error.value() != AisErrorCode.Success:
        print("Error uploading experiment: %s" % error.message())
        return

    error = handler.startUploadedExperiment(0)
    if error.value() != AisErrorCode.Success:
        print("Error starting experiment: %s" % error.message())
        return

In [7]:
def cp_mode(deviceName, current = 0.01, time = 10, repeat = 1, sampling_interval = 1):
    print("Device is Connected: %s" % deviceName)
    
    handler = tracker.getInstrumentHandler("Prime2645")

    # Connect signals; assigning the connection to "_" prevents printing the connection object.
    handler.activeDCDataReady.connect(lambda channel, data: print(f"timestamp:", "{:.3f},".format(data.timestamp),
                                                                  'voltage:', '{:.9f},'.format(data.counterElectrodeVoltage),
                                                                  "working Electrode Voltage:","{:.9f},".format(data.workingElectrodeVoltage),
                                                                  'counter electrode:', '{:.9f},'.format(data.counterElectrodeVoltage),
                                                                  'current:', "{:.9f}".format(data.current)))
    handler.experimentNewElementStarting.connect(lambda channel, data: print(f"New Node beginning:", data.stepName, "step number: ", data.stepNumber, " step sub : ", data.substepNumber))
    handler.experimentStopped.connect(lambda channel : print(f"Experiment Completed: %d" % channel))

    # Set up experiment elements
    experiment = AisExperiment()
    
    # define constant current experiment at current = current (defualt set at 0.01 A) with a sampling interval of 1 s and a duration of t s (defaults at 10 s)
    ccElement = AisConstantCurrentElement(current, sampling_interval, time)

    success = False
    subExperiment = AisExperiment()
    success &= subExperiment.appendElement(ccElement)
    success &= experiment.appendSubExperiment(subExperiment, repeat)

    error = handler.uploadExperimentToChannel(0, experiment)
    if error.value() != AisErrorCode.Success:
        print("Error uploading experiment: %s" % error.message())
        return

    error = handler.startUploadedExperiment(0)
    if error.value() != AisErrorCode.Success:
        print("Error starting experiment: %s" % error.message())
        return

In [ ]:
record_cv('Prime2645', low=0, high=0.5, end=0.5, step=0.01, sampling_interval=0.01)

In [ ]:
cp_mode("Prime2645", current = 0.01, time = 10)

In [ ]:
def reset_tracker():
    try:
        tracker.newDeviceConnected.disconnect()
    except Exception:
        pass  # No previous connection
    tracker.connectToDeviceOnComPort("COM7")
    tracker.newDeviceConnected.connect(cp_mode('Prime2645', current=0.001, time=0.1))

reset_tracker()

In [ ]:
import sys
import struct
import pandas as pd
from PySide6.QtWidgets import QApplication
from PySide6.QtCore import QEventLoop
from SquidstatPyLibrary import AisDeviceTracker, AisErrorCode, AisExperiment
from SquidstatPyLibrary import AisConstantPotElement, AisConstantCurrentElement, AisCyclicVoltammetryElement

def startExperiment_and_return_data(deviceName):
    print("Device is Connected:", deviceName)
    
    handler = tracker.getInstrumentHandler("Prime2645")
    
    # List to collect data from the experiment
    data_records = []
    
    # Connect signal to collect active DC data
    handler.activeDCDataReady.connect(
        lambda channel, data: data_records.append({
            'timestamp': round(data.timestamp, 3),
            'voltage': round(data.counterElectrodeVoltage, 9),
            'working_electrode_voltage': round(data.workingElectrodeVoltage, 9),
            'counter_electrode_voltage': round(data.counterElectrodeVoltage, 9),
            'current': round(data.current, 9)
        })
    )
    
    handler.experimentNewElementStarting.connect(
        lambda channel, data: print("New Node beginning:", data.stepName, 
                                      "step number:", data.stepNumber, "step sub:", data.substepNumber)
    )
    
    # Create a local event loop to wait for experiment completion
    loop = QEventLoop()
    handler.experimentStopped.connect(lambda channel: loop.quit())
    handler.experimentStopped.connect(lambda channel: print("Experiment Completed:", channel))
    
    # Set up experiment elements
    experiment = AisExperiment()
    cvElement = AisConstantPotElement(0.5, 0.1, 10)
    ccElement = AisConstantCurrentElement(0.1, 0.1, 10)
    cyclic = AisCyclicVoltammetryElement(0.0, -0.25, 0.25, 0.5, 0.1, 0.1)

    subExperiment = AisExperiment()
    success = True
    success &= subExperiment.appendElement(ccElement)
    success &= subExperiment.appendElement(cvElement)
    success &= subExperiment.appendElement(cyclic)
    success &= experiment.appendSubExperiment(subExperiment)

    error = handler.uploadExperimentToChannel(0, experiment)
    if error.value() != AisErrorCode.Success:
        print("Error uploading experiment:", error.message())
        return None

    error = handler.startUploadedExperiment(0)
    if error.value() != AisErrorCode.Success:
        print("Error starting experiment:", error.message())
        return None

    # Wait until the experiment stops
    loop.exec()

    # Create the inner DataFrame from collected records
    inner_df = pd.DataFrame(data_records)
    # Create the outer DataFrame with metadata and the inner DataFrame embedded
    outer_df = pd.DataFrame({
        'device': [deviceName],
        'experiment_data': [inner_df]
    })
    
    return outer_df

# ---------------------
# Setup code for Jupyter Notebook
# Use the Qt magic to integrate the event loop:

app = QApplication.instance() or QApplication(sys.argv)
tracker = AisDeviceTracker.Instance()
tracker.connectToDeviceOnComPort("COM5")
tracker.newDeviceConnected.connect(startExperiment)

# In a notebook, you don't call app.exec(); the %gui qt magic handles that.


In [ ]:
startExperiment_and_return_data("Prime2645")